In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
import KTBoost.KTBoost as KTBoost
from sklearn import linear_model
from sklearn.model_selection import KFold 
import sys

In [ ]:
# main_dir should be the direction to 
main_dir = sys.path[0]

dir_spotlift = main_dir + r'\spotlifts\BSTS_4min'
dir_X = main_dir + r'\X_matrix'

In [ ]:
def spotlift_group(X, y):
    df = X.copy()
    df['y']  = None
    for idx,row in df[['date_time','post_eff']].iterrows():
        dates = pd.date_range(start=row[0], end =row[1], freq='min')
        df.loc[idx, ['y']]= y[y['date_time'].isin(dates)].iloc[:,[1]].sum().values
    return df

In [ ]:
# final data selection
def prep_dataset(X_matrix, country, medium):
    filename = r'\lift_' + country + '_' + medium + '_case1_4min.csv'
    DataSet = pd.read_csv(dir_spotlift + filename, sep=';', index_col = 0, parse_dates=['time'])
    DataSet.columns = ['date_time', 'Spotlift']
    DataSet.loc[(DataSet['Spotlift']<0) | (DataSet['Spotlift'].isna()) , 'Spotlift']= 0
    return spotlift_group(X_matrix,DataSet[['date_time', 'Spotlift']])

X_nl = pd.read_csv(dir_X + '\X_nl_4min.csv', index_col='ad_group')
X_be = pd.read_csv(dir_X + '\X_be_4min.csv', index_col='ad_group')

NL_web = prep_dataset(X_nl, 'nl', 'web')
NL_app = prep_dataset(X_nl, 'nl', 'app')
BE_web = prep_dataset(X_be, 'be', 'web')
BE_app = prep_dataset(X_be, 'be', 'app')

In [ ]:
# Function to calculate the importances of Neural Nets using the Connection Weights approach
# Eventually, this is not used in our research------------------------------------------------------------------------------
def conn_weights(model):
    for layer in reversed(model.layers): 
        gew = layer.get_weights()[0]
        if gew.shape[1]==1:
            new_gew = gew
            continue
        else:
            new_gew = np.dot(gew,new_gew)
    return new_gew

# Function to display predictions for the different models that are used----------------------------------------------------
def disp_preds(model, X_train, X_test, y_train, y_test, new=False, ols= False):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
    if ols!=False:
        y_pred = np.dot(X_train,ols.params)
    else:
        y_pred = model.predict(X_train)
    if new:
        test = pd.DataFrame([y_train,y_pred]).transpose()
    else:
        test = pd.DataFrame([y_train,np.concatenate(y_pred,axis=0)]).transpose()
    test.columns =['real','pred']
    test.plot(ax=axes[0,0], style='.-')

    diff = test['real']-test['pred']
    diff.plot(ax=axes[0,1], style='.-')
    if ols!=False:
        y_pred = np.dot(X_test,ols.params)
    else:
        y_pred = model.predict(X_test)
    if new:
        test = pd.DataFrame([y_test,y_pred]).transpose()
    else:
        test = pd.DataFrame([y_test,np.concatenate(y_pred,axis=0)]).transpose()
    test.columns =['real','pred']
    test.plot(ax=axes[1,0], style='.-')

    diff = test['real']-test['pred']
    diff.plot(ax=axes[1,1], style='.-')
    plt.show()

In [ ]:
def neural_net(X_train,X_test,y_train,y_test, columns, disp):
    input_layer = X_train.shape[1]
    hidden_first=20
    hidden_second = 20
    act_fun = 'elu'
    act_fun_out = 'elu'
    bias = False
    metrics='MeanSquaredError'
    epochs = 18
    
    model = Sequential()
    # input layer
    model.add(Dense(hidden_first, input_dim=input_layer, activation=act_fun, use_bias=bias))
    # hidden layers
    model.add(Dense(hidden_second, activation=act_fun, use_bias=bias))
    model.add(Dense(hidden_second, activation=act_fun, use_bias=bias))
    # output layer
    model.add(Dense(1, activation=act_fun_out, use_bias=bias))
    model.compile(loss=metrics, optimizer='adam', metrics=['mean_squared_error'])

    # predict and plot including importances
    history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=epochs, verbose=0)
       
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    display('MSE Neural Network: ' + str(mse))
    
    if disp==True:
        # print the train and test MSE
        pd.DataFrame(history.history).drop(['loss', 'val_loss'],axis=1).plot()
        plt.show()
        # Plot the predictions
        disp_preds(model, X_train, X_test, y_train, y_test)
    
    # Importances---NOT USED
#     connection_weights = conn_weights(model)
#     frame = pd.DataFrame(connection_weights, index = columns)
#     frame_abs = frame.reindex(frame[0].abs().sort_values(ascending=False).index)
#     frame_norm = frame.sort_values(0, ascending=False)
#     print('neural importance')
#     display(frame_norm)
    return mse, mae
# --------------------------------------------------------------------------------------------------------------------------
def ols_reg(X_train,X_test,y_train,y_test, columns, disp):
    model = sm.OLS(y_train, X_train)
    results = model.fit()
#     print(results.summary(xname=columns))
    pred = np.dot(X_test,results.params)
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    display('MSE ols: ' + str(mse))
    
    if disp==True:
        # Plot the predictions
        disp_preds(model, X_train, X_test, y_train, y_test, True, results)
    return mse, mae
# --------------------------------------------------------------------------------------------------------------------------
def rf_reg(X_train,X_test,y_train,y_test, columns, disp = False):
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    display('MSE Random Forest: ' + str(mse))
    
    if disp==True:
        # Plot the predictions
        disp_preds(rf, X_train, X_test, y_train, y_test, True)
    return mse,mae
# --------------------------------------------------------------------------------------------------------------------------
def tobit_reg(X_train,X_test,y_train,y_test, columns, disp):
    model = KTBoost.BoostingRegressor(loss='tobit',yl=10^-10,yu=10000)

    # Train model
    results = model.fit(X_train,y_train)

    # Make prediction
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    print('MSE Tobit : ' + str(mse))

    if disp==True:
        # Plot the predictions
        disp_preds(model, X_train, X_test, y_train, y_test, True)
    return mse,mae
# --------------------------------------------------------------------------------------------------------------------------
def lasso_reg(X_train,X_test,y_train,y_test, columns, disp):
    model = linear_model.ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                                      fit_intercept=False, normalize=True,random_state=0)
    results = model.fit(X_train,y_train)
    coefs = pd.DataFrame({'Coefficient': results.coef_}, index=columns)
    coefs = coefs.reindex(coefs.Coefficient.abs().sort_values(ascending=False).index)
    display(coefs)
    
    pred = model.predict(X_test)
    pred_train = model.predict(X_train)
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    print('MSE Lasso : ' + str(mse))
    print('R2 score: ' + str(r2_score(y_train, pred_train)))
    print('l1-ratio: ' + str(model.l1_ratio_))
    print('Alpha: '+str(results.alpha_))
    
    if disp==True:
        # Plot the predictions
        disp_preds(model, X_train, X_test, y_train, y_test, True)
    return mse,mae
# --------------------------------------------------------------------------------------------------------------------------
def compares(X_train,X_test,y_train,y_test, columns, models, disp):
    mse_rf, mse_ols, mse_tobit, mse_ann, mse_lasso = None,None,None,None,None
    mae_rf, mae_ols, mae_tobit, mae_ann, mae_lasso = None,None,None,None,None
    # Random Forest-------------------------------------------------------------------
    if 'rf' in models:
        mse_rf, mae_rf = rf_reg(X_train,X_test,y_train,y_test, columns, disp)
    # OLS regression------------------------------------------------------------------
    if 'ols' in models:
        mse_ols, mae_ols = ols_reg(X_train,X_test,y_train,y_test, columns, disp)
    # Tobit---------------------------------------------------------------------------
    if 'tobit' in models:
        mse_tobit, mae_tobit = tobit_reg(X_train,X_test,y_train,y_test, columns, disp)
    # Neural network------------------------------------------------------------------
    if 'ann' in models:
        mse_ann, mae_ann = neural_net(X_train,X_test,y_train,y_test, columns, disp)
    # Lasso---------------------------------------------------------------------------
    if 'lasso' in models:
        mse_lasso, mae_lasso = lasso_reg(X_train,X_test,y_train,y_test, columns, disp)
    return [mse_rf, mse_ols, mse_tobit, mse_ann, mse_lasso,
            mae_rf, mae_ols, mae_tobit, mae_ann, mae_lasso]

In [ ]:
def ANN(data, scaler, drop_col=None, kfold=True, models = ['lasso'], disp=False):
    #set random seed/state
    seed_value = 309
    tf.random.set_seed(seed_value)
    
    # normalize
    X = data.drop(['date_time', 'y', 'post_eff'], axis=1)

    if drop_col is not None:
        X = X.drop(drop_col,axis=1)
    columns = X.columns.tolist()

    if scaler is not None:
        if scaler== 'standard':
            sc = StandardScaler()
        elif scaler =='minmax':
            sc = MinMaxScaler()
        X = sc.fit_transform(X.values)
    else:
        X = X.to_numpy()
    
    # Add constant
    X = sm.add_constant(X)    
    columns.insert(0,'const')

    y = data['y'].values
    y = np.asarray(y).astype('float32')

    if kfold:
        # K-fold splits
        kf = KFold(n_splits=5, random_state=456, shuffle=True)

        temp = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            temp.append(compares(X_train,X_test, y_train, y_test, columns, models, disp))
    else:
        temp = []
        # If it is not a k-fold, we use all the data to get the final model, we do not test anymore        
        temp.append(compares(X,X, y, y, columns, models, disp))
        
    results = pd.concat([pd.DataFrame([i], columns=['rf_mse', 'ols_mse', 'tobit_mse', 'ann_mse', 'lasso_mse',
                                                    'rf_mae', 'ols_mae', 'tobit_mae', 'ann_mae', 'lasso_mae']) for i in temp], ignore_index=True)
    display(results)
    return results

In [ ]:
# Table 7: Model selection
# To avoid multicollinearity, of each category, one feature is left out. Prime and the operators are not used anyway
columns = ['operator_Talpa TV',
           'operator_Ad Alliance',
           'operator_Ster',
           'prod_cat_wasmachines',  
           'prime', 
           'duration_30 + 10 + 5', 
           'channel_Veronica',
           'pos_break_Any Other Position']
models = ['rf', 'ann', 'ols', 'tobit']
results = ANN(NL_web, None, drop_col=columns, kfold =True, models = models, disp=False)

display(results.mean(), results.std())

In [ ]:
# Tabel 8,26 and 27: Elastic Net features the Netherlands
columns = ['operator_Talpa TV',
           'operator_Ad Alliance',
           'operator_Ster',
           'prime']
results_nl_web = ANN(NL_web, None, drop_col=columns, kfold =False, models = ['lasso'], disp=False)
results_nl_app = ANN(NL_app, None, drop_col=columns, kfold =False, models = ['lasso'], disp=False)

In [ ]:
# Tabel 28 and 29: Elastic Net features Belgium
columns = ['operator_MEDIALAAN',
           'operator_SBS',
           'operator_TRANSFER',
           'prime']
results_be_web = ANN(BE_web, None, drop_col=columns, kfold =False, models = ['lasso'], disp=False)
results_be_app = ANN(BE_app, None, drop_col=columns, kfold =False, models = ['lasso'], disp=False)